


MLIR

Refinement type system

`SeqSort(Int) -> Real` But with refinement types.
Reshaping scheduling operations as transformations on this.


python sexp frontend into xdsl?

https://arxiv.org/abs/1912.13451  Introduction to Rank-polymorphic Programming in Remora (Draft)

https://github.com/remora-lang/interpreter


Dialegg - where are dialegg's rules?
eqsat



"scheduling"
Large and small scale


smtlib macros

(a -> b) -> c
(int -> int) -> Real  (a sequuence of integer. Tail of sequence is ignored?)

melior 
jlm rvsdg dialect. What makes rvsdg different from mlir itself?


gradual typing


# F2
https://arxiv.org/pdf/2505.23819 Linear Layouts: Robust Code Generation of Efficient Tensor Computation Using F2
https://link.springer.com/article/10.1007/s11786-025-00602-8

# remora

is (array [] 17) the same as (array [1] 17)


# interpreter

In [ ]:
import re
tokens = [
    ("WS", r"\s+"), 
    ("COMMENT", r"%[^\n\r]*"),
    ("LPAREN", r"\("),
    ("RPAREN", r"\)"),
    ("COMMA", r","),
    ("PERIOD", r"\."),
    ("IMPLY", r":-"),
    ("IDENT", r"[A-Za-z\-!=\+\*\_<>][A-Za-z0-9\-!=\+\*\_<>]*"),
    ("NUMBER", r"-?\d+(?:\.\d+)?")
    #("QUOTE", r"\|")
]

regex = "|".join(f"(?P<{name}>{regex})" for name,regex in tokens)
pattern = re.compile(regex)
def parse(s : str):
    it = pattern.finditer(s)
    def args():
        items = []
        for match_ in it:
            match match_.lastgroup:
                case "WS" | "COMMENT":
                    continue
                case "IDENT" | "NUMBER":
                    items.append([match_.group()] + expr())
                case "COMMA":
                    continue
                case "RPAREN":
                    return items
                case _:
                    raise ValueError(f"unexpected token {match_.group()}")
        raise ValueError("unbalanced paren")
    def expr():
        for match_ = next(it):
            match match_.lastgroup:
                case "WS" | "COMMENT":
                    continue
                case "IDENT":
                    ident = match_.group()
                    match_ = next(it)
                    if match_.lastgroup == "LPAREN":
                        return (ident, args())
        raise ValueError("unbalanced paren")
    # match sexp*
    out = []
    for match_ in it:
        match match_.lastgroup:
            case "WS" | "COMMENT":
                continue
            case _:
                out.append(expr())
    return out

In [38]:

p = iter(range(5))
for x in p:
    print("x", x)
    for y in p:
        print("y", y)

x 0
y 1
y 2
y 3
y 4


https://norvig.com/lispy.html
https://github.com/dhilst/lis.py
https://github.com/pavpanchekha/pylisp
Make a numpy interpreter
jax / pytorch
compile to xdsl
compile to llvm
compile to smt

In [8]:
from kdrag.parsers.sexp import parse
import operator
import functools
import numpy as np
parse("(+ 1 2 3 4)")

def expr(env, sexp):
    match sexp:
        case int() | float():
            return sexp
        case str():
            return env[sexp]
        case ["if", cond, then, otherwise]:
            return expr(env, then) if expr(env, cond) else expr(env, otherwise)
        case ["set!", name, value]:
            env[name] = expr(env, value)
        case ["lambda", [*args], body]:
            return lambda *argvals: expr({**env, **dict(zip(args, argvals))}, body)
        case ["quote", body]:
            return body
        case ["define-macro", name, [*args], body]:
            env["__macros"][name] = lambda *argvals: expr({**env, **dict(zip(args, argvals))}, body)
        case ["eval", body]:
            return expr(env, expr(env, body))
        case [".", obj, attr]:
            return getattr(expr(env, obj), attr) # attr is intrinsically quoted?
        case ["array", shape, *vals]:
            return np.array(vals).reshape(shape)
        case ["frame", shape, *vals]:
            raise NotImplementedError()
        case [op, *args]:
            if isinstance(op, str) and op in env["__macros"]:
                return expr(env, env["__macros"][op](*args))
            else:
                return expr(env, op)(*(expr(env, arg) for arg in args))

def stmt(env, sexp):
    match sexp:
        case ["define", name, value]:
            env[name] = expr(env, value)
        case _:
            print(expr(env, sexp))

def stmts(sexps):
    def myassert(x):
        assert x
    env = {"+": lambda *args: functools.reduce(operator.add, args),
           "print": print,
           "assert": myassert,
           "nil" : [],
           "true": True,
           "false": False,
           "-": operator.sub,
           "*": lambda *args: functools.reduce(operator.mul, args),
           "=": operator.eq,
           "<": operator.lt,
           ">": operator.gt,
           "<=": operator.le,
           ">=": operator.ge,
           "/": operator.truediv,
           "car": lambda x: x[0],
           "cdr": lambda x: x[1:],
           "cons": lambda x, xs: [x] + xs,
           "list": lambda *args: list(args),
           "map": lambda f, lst: list(map(f, lst)),
           "array": lambda shape, *vals: np.array(vals).reshape(shape),
           "reduce": lambda f, arr: f.reduce(arr),
           "np": np,
           "__macros" : {}
           }
    for sexp in sexps:
        stmt(env, sexp)
    return env

stmts(parse("""
(define x 10)
(define y 20)
(+ x y 30)
(print 17)
(assert true)
;(assert false)
((lambda (x) (+ x 1)) 10)
(+ (quote (1 2)) (quote (3 4)))
(cdr (quote (1 2 3 4)))
(car (quote (1 2 3 4)))
(map (lambda (x) (* x 2)) (quote (1 2 3 4 5 6 7 8 9 10)))
(eval (quote (+ 1 2)))
(array (2 5) 1 2 3 4 5 6 7 8 9 10)
(array () 17)
((. np sum) (array (2 5) 1 2 3 4 5 6 7 8 9 10))
(define-macro when (cond body) (list (quote if) cond body (quote nil)))
(when false (print (quote it_worked)))

"""))


60
17
None
None
11
[1, 2, 3, 4]
[2, 3, 4]
1
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
3
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]]
17
55
None
[]


{'+': <function __main__.stmts.<locals>.<lambda>(*args)>,
 'print': <function print(*args, sep=' ', end='\n', file=None, flush=False)>,
 'assert': <function __main__.stmts.<locals>.myassert(x)>,
 'nil': [],
 'true': True,
 'false': False,
 '-': <function _operator.sub(a, b, /)>,
 '*': <function __main__.stmts.<locals>.<lambda>(*args)>,
 '=': <function _operator.eq(a, b, /)>,
 '<': <function _operator.lt(a, b, /)>,
 '>': <function _operator.gt(a, b, /)>,
 '<=': <function _operator.le(a, b, /)>,
 '>=': <function _operator.ge(a, b, /)>,
 '/': <function _operator.truediv(a, b, /)>,
 'car': <function __main__.stmts.<locals>.<lambda>(x)>,
 'cdr': <function __main__.stmts.<locals>.<lambda>(x)>,
 'cons': <function __main__.stmts.<locals>.<lambda>(x, xs)>,
 'list': <function __main__.stmts.<locals>.<lambda>(*args)>,
 'map': <function __main__.stmts.<locals>.<lambda>(f, lst)>,
 'array': <function __main__.stmts.<locals>.<lambda>(shape, *vals)>,
 'reduce': <function __main__.stmts.<locals>.<lambd

In [2]:
parse("""
(define x 10)
(define y 20)
(+ x y 30)
(print 17)
(assert true)
;(assert false)
((lambda (x) (+ x 1)) 10)
(+ (quote (1 2)) (quote (3 4)))
(cdr (quote (1 2 3 4)))
(car (quote (1 2 3 4)))
(map (lambda (x) (* x 2)) (quote (1 2 3 4 5 6 7 8 9 10)))
(eval (quote (+ 1 2)))
(array (2 5) 1 2 3 4 5 6 7 8 9 10)
(array () 17)
((. np sum) (array (2 5) 1 2 3 4 5 6 7 8 9 10))

""")

[['define', 'x', 10],
 ['define', 'y', 20],
 ['+', 'x', 'y', 30],
 ['print', 17],
 ['assert', 'true'],
 [['lambda', ['x'], ['+', 'x', 1]], 10],
 ['+', ['quote', [1, 2]], ['quote', [3, 4]]],
 ['cdr', ['quote', [1, 2, 3, 4]]],
 ['car', ['quote', [1, 2, 3, 4]]],
 ['map',
  ['lambda', ['x'], ['*', 'x', 2]],
  ['quote', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]],
 ['eval', ['quote', ['+', 1, 2]]],
 ['array', [2, 5], 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 ['array', [], 17],
 [['np', 'sum'], ['array', [2, 5], 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]]

In [273]:
np.sum

<function sum at 0x760ee021d030>

In [ ]:
def expr(env, builder, sexp):
    

In [ ]:
import numpy as np

np.ufunc.reduce()

typed assembly language approach to bytecode?
Kind of an odd idea
overloading, tracing, ast sweep, bytecode, weird frame stuff (?), 

full sweep


# dep types
My stuff
smt dialect go into attributes.. yada yada dependent mlir?
good blog post
https://rschenck.com/docs/array-props.pdf  Verifying Properties of Index Arrays in a Purely-Functional Data-Parallel Language


# cost models
cache modeling
scallop neuro learning of

Roughly model of pipes?

https://dl.acm.org/doi/10.1145/3656452 Falcon: A Scalable Analytical Cache Model


Unison had a model.

  resource
insn(resource, time, )

Given programn, write prolog program to compute the time it would take.

pipeline_state()
cache_state(t, )



# futhark


# klr
https://github.com/leanprover/KLR
https://github.com/leanprover/TensorLib

In [207]:
! klr

klr [0.0.12]
KLR is an IR for NKI and other tensor-like languages in Lean.

USAGE:
    klr [SUBCOMMAND] [FLAGS]

FLAGS:
    -h, --help  Prints this message.
    --version   Prints the version.

SUBCOMMANDS:
    compile     Compile a NKI kernel
    eval-klr    Evaluate a kernel using a pure-Lean KLR interpreter. Outputs one
                npy file for each output.
    gather      Gather Python sources into an AST file
    info        Display information about a KLR file
    nki-to-klr  Compile NKI kernel to KLR
    trace       Trace Python to KLR
    typecheck   Run the type checker on a Python AST file


In [214]:
import klr.frontend as fe
def kernel_function(a, b):
  return a + b
kernel = fe.Kernel(kernel_function)
kernel.specialize(1, 2)
# Serialize the result
ba = kernel.serialize("/tmp/kernel.klr")
  # get a KLR abstract snytax tree
import klr.ast_nki
ast = fe.deserialize(ba)
ast

Kernel(entry='__main__.kernel_function', funs=[Fun(name='__main__.kernel_function', file='unknown', line=2, body=[Stmt(stmt=Stmt_ret(e=Expr(expr=Expr_binOp(op=<BinOp.Add: 9>, left=Expr(expr=Expr_var(name='a'), pos=Pos(line=2, column=9, lineEnd=2, columnEnd=10)), right=Expr(expr=Expr_var(name='b'), pos=Pos(line=2, column=13, lineEnd=2, columnEnd=14))), pos=Pos(line=2, column=9, lineEnd=2, columnEnd=14))), pos=Pos(line=2, column=2, lineEnd=2, columnEnd=14))], args=[Param(name='a', dflt=None), Param(name='b', dflt=None)])], args=[Arg(name='b', value=Expr(expr=Expr_value(value=Value_int(value=2)), pos=Pos(line=0, column=0, lineEnd=None, columnEnd=None))), Arg(name='a', value=Expr(expr=Expr_value(value=Value_int(value=1)), pos=Pos(line=0, column=0, lineEnd=None, columnEnd=None)))], globals=[])

In [213]:
! cat /tmp/kernel.klr

����  �� �cKLR���� �x__main__.kernel_function��
 �x__main__.kernel_functiongunknown�� ��� ������ ���aa�d �	������
�������d �	�������d ���������	 �aa�� ��	 �ab�� ��� �ab� �� ����d �  �� ��� �� �aa� �� ����d �  �� ��� ��

# llvm



llvmlite vs llvmpy


In [ ]:
module = ir.Module(name="testo")
# and declare a function named "fpadd" inside it
func = ir.Function(module, fnty, name="fpadd")

# Now implement the function
block = func.append_basic_block(name="entry")
builder = ir.IRBuilder(block)
a, b = func.args
result = builder.fadd(a, b, name="res")
builder.ret(result)

# Print the module IR
print(module)

; ModuleID = "testo"
target triple = "unknown-unknown-unknown"
target datalayout = ""

define double @"fpadd"(double %".1", double %".2")
{
entry:
  %"res" = fadd double %".1", %".2"
  ret double %"res"
}



In [ ]:
from kdrag.all import *
import llvmlite
def of_sort(s : smt.SortRef) -> llvmlite.ir.Type:
    if s == smt.BoolSort():
        return llvmlite.ir.IntType(1)
    elif isinstance(s, smt.BitVecSortRef):
        return llvmlite.ir.IntType(s.size())
    else:
        raise ValueError(f"unsupported sort {s}")

of_sort(smt.BitVecSort(32))

def of_expr(e : smt.ExprRef, env, builder) -> llvmlite.ir.Value:
    if smt.is_app(e):
        decl = e.decl()
        children = e.children()
        if decl.name() == "bvadd":
            lhs, rhs = (of_expr(c, env, builder) for c in children)
            return builder.add(lhs, rhs)
        elif smt.is_const(e):
            if smt.is_bv(e):
                return llvmlite.ir.Constant(of_sort(e.sort()), e.as_long())
        else:
            raise ValueError(f"unsupported decl {decl}")
    else:
        raise ValueError(f"unsupported expr {e}")

def func(name, args, e : smt.ExprRef):
    module = ir.Module(name="testo")
    fnty = ir.FunctionType(of_sort(e.sort()), [of_sort(a) for a in args])
    func = ir.Function(module, fnty, name=name)

    # Now implement the function
    block = func.append_basic_block(name="entry")
    builder = ir.IRBuilder(block)
    env = dict(zip(args, func.args))
    retval = of_expr(e, env, builder)
    builder.ret(retval)
    return module





# mlir

https://github.com/opencompl/lean-mlir?tab=readme-ov-file

"just" printing the string.

https://gitlab.inria.fr/snoiry/mlir-list list lang tutorial

https://www.reddit.com/r/Compilers/comments/1nrj7gu/mlir_tutorial/
Repository with the code: https://github.com/rafasumi/mlir-tutorial
Lecture notes: https://github.com/rafasumi/mlir-tutorial/blob/main/assets/MLIR_tutorial_SBLP_2025.pdf

sum all array
vector add two arrays
blur
transpose (?)


In [180]:
%%file /tmp/myadd.mlir

module {
    func.func @myadd(%x : i32, %y : i32) -> i32 {
        %z = arith.addi %x, %y : i32
        return %z : i32
    }
}


Writing /tmp/myadd.mlir


In [182]:
! mlir-opt-20 /tmp/myadd.mlir -convert-to-llvm -o /tmp/add_llvm.mlir && cat /tmp/add_llvm.mlir

module {
  llvm.func @myadd(%arg0: i32, %arg1: i32) -> i32 {
    %0 = llvm.add %arg0, %arg1 : i32
    llvm.return %0 : i32
  }
}



Hmm. Maybe I want an interpreter to z3


In [258]:
from xdsl.ir import Block, Region
from xdsl.printer import Printer
Block()

blk = Block(arg_types=[IntegerType(32)])
blk.args
builder = Builder(InsertPoint.at_end(blk))
i42 = builder.insert(ConstantOp(IntegerAttr(42, IntegerType(64))))
builder.insert(arith.AddiOp(blk.args[0], i42))
Printer().print(blk)


^0(%0 : i32):
  %1 = arith.constant 42 : i64
  %2 = arith.addi %0, %1 : i32


/tmp/ipykernel_323901/3024886619.py:10: DeprecationWarning: Please use type-specific print methods
  Printer().print(blk)


In [242]:
import xdsl.dialects.smt as smt
from xdsl_smt.traits.smt_printer import print_to_smtlib
dir(smt)
smt.BoolType()
smt.BitVectorType(32)
smt.FuncType([smt.BoolType()], smt.BoolType())
#print_to_smtlib()
const = smt.BvConstantOp(smt.BitVectorAttr(42,32))
import io

    # Create an empty StringIO object
string_buffer = io.StringIO()
print_to_smtlib(ModuleOp([const]), string_buffer)
string_buffer.getvalue()


'(declare-datatypes ((Pair 2)) ((par (X Y) ((pair (first X) (second Y))))))\n'

In [124]:
import xdsl
from xdsl.builder import Builder, InsertPoint
from xdsl.dialects.arith import AddiOp
from xdsl.dialects.builtin import ModuleOp, IntegerType, IntegerAttr
from xdsl.dialects.arith import ConstantOp

c0 = ConstantOp(IntegerAttr(0, 64))
c0
addi = AddiOp(c0.result, c0.result)
ModuleOp([addi])

ModuleOp(_operands=(), _operand_uses=(), results=(), _successors=(), _successor_uses=(), properties={}, attributes={}, regions=(Region(num_blocks=1),))

In [178]:
import xdsl.dialects.builtin as builtin
dir(builtin)
builtin.Block

xdsl.ir.core.Block

In [176]:
dir(arith)

['AddUIExtendedOp',
 'AddfOp',
 'AddiOp',
 'AndIOp',
 'AnyAttr',
 'AnyFloat',
 'AnyFloatConstr',
 'AnyOf',
 'Arith',
 'Attribute',
 'BitEnumAttribute',
 'BitcastOp',
 'CMPF_COMPARISON_OPERATIONS',
 'CMPI_COMPARISON_OPERATIONS',
 'CeilDivSIOp',
 'CeilDivUIOp',
 'ClassVar',
 'CmpfOp',
 'CmpiHasCanonicalizationPatterns',
 'CmpiOp',
 'Commutative',
 'ComparisonOperation',
 'ConditionallySpeculatable',
 'ConstantLike',
 'ConstantOp',
 'ContainerOf',
 'DenseIntOrFPElementsAttr',
 'DenseResourceAttr',
 'Dialect',
 'DivSIOp',
 'DivUIOp',
 'DivUISpeculatable',
 'DivfOp',
 'ExtFOp',
 'ExtSIOp',
 'ExtUIOp',
 'FPToSIOp',
 'FPToUIOp',
 'FastMathAttrBase',
 'FastMathFlag',
 'FastMathFlagsAttr',
 'FixedBitwidthType',
 'Float16Type',
 'Float32Type',
 'Float64Type',
 'FloatAttr',
 'FloatingPointLikeBinaryOpHasCanonicalizationPatternsTrait',
 'FloatingPointLikeBinaryOpHasFastReassociativeCanonicalizationPatternsTrait',
 'FloatingPointLikeBinaryOperation',
 'FloatingPointToIntegerBaseOp',
 'FloorDivSIOp'

In [ ]:
from xdsl.context import Context
from xdsl.parser import Parser
from xdsl.ir import OpResult
from xdsl.dialects import builtin, func, arith, scf
ctx = Context()
ctx.load_dialect(builtin.Builtin)
ctx.load_dialect(func.Func)
ctx.load_dialect(scf.Scf)
ctx.load_dialect(arith.Arith)

module_text = """
builtin.module {
  func.func @sum_of_squares(%n : index) -> index {
    %zero = arith.constant 0 : index
    %one = arith.constant 1 : index
    %res = scf.for %i = %zero to %n step %one iter_args(%acc_in = %zero) -> (index) {
      %square = arith.muli %i, %i : index
      %acc_out = arith.addi %acc_in, %square : index
      scf.yield %acc_out : index
    }
    func.return %res : index
  }
}
"""

module = Parser(ctx, module_text).parse_module()
module
print([op.name for op in module.walk()])

# PassPipeline

['builtin.module', 'func.func', 'arith.constant', 'arith.constant', 'scf.for', 'arith.muli', 'arith.addi', 'scf.yield', 'func.return']


In [269]:
modtext = r"""
module {
  func.func @is_positive(%x: i32) -> i32 {
    %c0 = arith.constant 0 : i32
    %cond = arith.cmpi sgt, %x, %c0 : i32
    %res = scf.if %cond -> (i32) {
      %c1 = arith.constant 1 : i32
      scf.yield %c1 : i32
    } else {
      scf.yield %c0 : i32
    }
    return %res : i32
  }
}
"""

module = Parser(ctx, modtext).parse_module()
print(module)
module.verify()
import xdsl.transforms.convert_scf_to_cf as scf2cf

scf2cf.ConvertScfToCf().apply(ctx, module)
module.verify()
print(module)

builtin.module {
  func.func @is_positive(%x : i32) -> i32 {
    %c0 = arith.constant 0 : i32
    %cond = arith.cmpi sgt, %x, %c0 : i32
    %res = scf.if %cond -> (i32) {
      %c1 = arith.constant 1 : i32
      scf.yield %c1 : i32
    } else {
      scf.yield %c0 : i32
    }
    func.return %res : i32
  }
}
builtin.module {
  func.func @is_positive(%x : i32) -> i32 {
    %c0 = arith.constant 0 : i32
    %cond = arith.cmpi sgt, %x, %c0 : i32
    cf.cond_br %cond, ^0, ^1
  ^0:
    %c1 = arith.constant 1 : i32
    cf.br ^2(%c1 : i32)
  ^1:
    cf.br ^2(%c0 : i32)
  ^2(%res : i32):
    cf.br ^3
  ^3:
    func.return %res : i32
  }
}


In [237]:
# https://github.com/xdslproject/xdsl/blob/main/docs/Toy/toy/compiler.py
ctx = Context()
from xdsl.dialects import builtin, func, arith, scf, cf, affine
from xdsl.transforms.canonicalize import CanonicalizePass
from xdsl.transforms.dead_code_elimination import DeadCodeElimination
ctx.load_dialect(affine.Affine)
ctx.load_dialect(arith.Arith)
ctx.load_dialect(builtin.Builtin)
ctx.load_dialect(func.Func)
ctx.load_dialect(scf.Scf)
ctx.load_dialect(cf.Cf)
mod = Parser(ctx, r"""
module {
  func.func @main(%c: i1) -> i32 {
    scf.if %c {
      %x = arith.constant 0 : i32
      return %x : i32
    } else {
      %y = arith.constant 1 : i32
      return %y : i32
    }
  }
}
""").parse_op()
mod.verify()
print(mod)
CanonicalizePass().apply(ctx, mod)
print(mod)
mod.verify()

VerifyException: Operation scf.if terminates block in single-block region but is not a terminator

In [147]:
mod.body.block

Block(_args=(), num_ops=0)

In [153]:
mod = ModuleOp([])
builder = Builder(InsertPoint.at_end(mod.body.block))
builder.insert(ConstantOp(IntegerAttr(42, 64)))

ConstantOp(_operands=(), _operand_uses=(), results=(<OpResult[i64] index: 0, operation: arith.constant, uses: 0>,), _successors=(), _successor_uses=(), properties={'value': IntegerAttr(value=IntAttr(data=42), type=IntegerType(64))}, attributes={}, regions=())

In [162]:
print(mod)

builtin.module {
  %0 = arith.constant 42 : i64
}


In [163]:
%%file /tmp/add.mlir
module {
  func.func @main() -> i32 {
    %c2 = arith.constant 2 : i32
    %c3 = arith.constant 3 : i32
    %sum = arith.addi %c2, %c3 : i32
    return %sum : i32
  }
}


Writing /tmp/add.mlir


In [175]:
! mlir-opt-20 /tmp/add.mlir \
  -convert-to-llvm \
  -reconcile-unrealized-casts \
  -o /tmp/add_llvm.mlir

In [173]:
! cat /tmp/add_llvm.mlir

module {
  llvm.func @main() -> i32 {
    %0 = llvm.mlir.constant(2 : i32) : i32
    %1 = llvm.mlir.constant(3 : i32) : i32
    %2 = llvm.mlir.constant(5 : i32) : i32
    llvm.return %2 : i32
  }
}



In [174]:
! mlir-translate-20 --mlir-to-llvmir /tmp/add_llvm.mlir -o /tmp/add.ll

## linalg
https://www.stephendiehl.com/posts/mlir_linear_algebra/
https://mlir.llvm.org/docs/Dialects/Linalg/
https://mlir.llvm.org/docs/Tutorials/transform/Ch0/

In [ ]:
%%file /tmp/matmul.mlir


module {
    func.func @mymatmul(%A: memref<4x4xf32>, %B: memref<4x4xf32>, %C: memref<4x4xf32>) {
        linalg.matmul 
            ins(%A, %B : memref<4x4xf32>, memref<4x4xf32>)
            outs(%C : memref<4x4xf32>) -> memref<4x4xf32>
        return 
    }
}


Overwriting /tmp/matmul.mlir


In [15]:
%%file /tmp/matmul.mlir

module {
    func.func @matmul(
    %A: memref<8x10xf32>,
    %B: memref<10x16xf32>,
    %C: memref<8x16xf32>
) {
    linalg.matmul ins(%A, %B : memref<8x10xf32>, memref<10x16xf32>) outs(%C : memref<8x16xf32>)
    return
}
}

Overwriting /tmp/matmul.mlir


-convert-linalg-to-loops  -convert-linalg-to-affine

In [16]:
! mlir-opt-20 /tmp/matmul.mlir -convert-linalg-to-loops -o /tmp/matmul_loops.mlir && cat /tmp/matmul_loops.mlir

module {
  func.func @matmul(%arg0: memref<8x10xf32>, %arg1: memref<10x16xf32>, %arg2: memref<8x16xf32>) {
    %c0 = arith.constant 0 : index
    %c8 = arith.constant 8 : index
    %c1 = arith.constant 1 : index
    %c16 = arith.constant 16 : index
    %c10 = arith.constant 10 : index
    scf.for %arg3 = %c0 to %c8 step %c1 {
      scf.for %arg4 = %c0 to %c16 step %c1 {
        scf.for %arg5 = %c0 to %c10 step %c1 {
          %0 = memref.load %arg0[%arg3, %arg5] : memref<8x10xf32>
          %1 = memref.load %arg1[%arg5, %arg4] : memref<10x16xf32>
          %2 = memref.load %arg2[%arg3, %arg4] : memref<8x16xf32>
          %3 = arith.mulf %0, %1 : f32
          %4 = arith.addf %2, %3 : f32
          memref.store %4, %arg2[%arg3, %arg4] : memref<8x16xf32>
        }
      }
    }
    return
  }
}



Affine manipulations

In [ ]:

from kdrag.all import *

# integer set
Loop2 = smt.ArraySort(smt.IntSort(), smt.IntSort(), smt.BoolSort())

# loop swapping

def Interval(start,end):
    return smt.Lambda([i], smt.And(start <= i, i < end))

smt.Lambda([i,j], Interval(0,10)(i) * Interval(0,i)[j])

Family = smt.ArraySort(smt.IntSort(), smt.ArraySort(smt.IntSort(), smt.IntSort()))

smt.Tuple(smt.IntSort(), smt.IntSort())


def is_family(): ...
def is_set(): ...

def lift(s1 : Family) -> Loop2 : # uncurry
    return smt.Lambda([i,j], s[i][j])




